In [1]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

from tqdm import tqdm


In [2]:
def bs_object_from_url(url) : 
    response = requests.get(url,
        headers={'User-Agent' : 'Mozilla 5.0'}
    )

    bs = BeautifulSoup(response.text,'lxml')
    
    return bs

def get_contents_from_rul(url) :
    bs = bs_object_from_url(url)

    if bs.select('#contents') :
        return bs_object_from_url(url).select('#contents')[0].text.strip()

    if bs.select('.content') :
        return bs_object_from_url(url).select('.content')[0].text.strip()

    if bs.select('#dic_area') :
        return bs_object_from_url(url).select('#dic_area')[0].text.strip()

    return -1

In [3]:
def crawl_news(date,oid,page) : 
    for page in range(1,last_page_num + 1) :
        url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date={date}&page={page}'
        bs_page = bs_object_from_url(url)
    
        titles_and_content_links = bs_page.select('dt > a')[1::2]
        titles = [titles_and_content_link.text.strip() for titles_and_content_link in titles_and_content_links]
        content_url = [titles_and_content_link.get('href') for titles_and_content_link in titles_and_content_links]
        contents = [get_contents_from_rul(url) for url in content_url]


        temps = pd.DataFrame({ 
            'date': date,
            'press_name': oid,
            'title': titles,
            'content': contents,
            'content_url': content_url
        })

        return temps

In [4]:
news = pd.DataFrame()
for oid in tqdm(('023','020')) :
    for date in pd.date_range('20221126','20221127') :
        date = date.strftime("%Y%m%d")
        
        last_pase_url = f"https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date={date}&page=100000"
        bs = bs_object_from_url(last_pase_url)
        
        last_page_num = int(bs.select('div.paging strong')[0].text.strip())

        for page in range(1,last_page_num + 1) :
            temp = crawl_news(date,oid,page)

            news = news.append(temp)

  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_1880\1433019421.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news = news.append(temp)
C:\Users\user\AppData\Local\Temp\ipykernel_1880\1433019421.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news = news.append(temp)
C:\Users\user\AppData\Local\Temp\ipykernel_1880\1433019421.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news = news.append(temp)
C:\Users\user\AppData\Local\Temp\ipykernel_1880\1433019421.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news = news.append(temp)
 50%|█████     | 1/2 [00:43<00:43, 43.50s/it]C:\Users\user\

In [5]:
news

,date,press_name,title,content,content_url
0,20221126,023,[NOW] 시의원에 ‘사과문 모욕’ 준 민노총,운영실태 지적당하자 서체·크기 콕 찍으며 “이대로 사과문 쓰라”서울시의회 “反민주주...,https://n.news.naver.com/mnews/article/023/000...
1,20221126,023,화물연대 운송 거부 지속땐... 대통령실 “29일 업무개시명령 검토”,민노총 화물연대 총파업 돌입시멘트·정유 화물차 조합원들 운송 거부땐 화물면허 취소돼...,https://n.news.naver.com/mnews/article/023/000...
2,20221126,023,"손흥민·호날두 괴롭힌 헐거운 완장, ‘싸구려’ 논란에 결국…",지난 24일 한국과 우루과이의 2022 카타르 월드컵 조별리그 H조 1차전에서 손흥...,https://n.news.naver.com/mnews/article/023/000...
3,20221126,023,"“이란 선수들, 월드컵 후 처형당할 수도” 외신 보도, 왜?",지난 21일 이란과 잉글랜드 경기가 시작되기 전 이란 선수들이 국가 제창을 거부하고...,https://n.news.naver.com/mnews/article/023/000...
4,20221126,023,광화문에선 ‘태극기 모자’ 남대문에선 ‘LED 촛불’… 1.5km 사이 열린 진보·...,토요일인 11월 26일 서울 세종대로에서 열린 보수단체 집회(왼쪽)와 서울 태평로 ...,https://n.news.naver.com/mnews/article/023/000...
...,...,...,...,...,...
15,20221127,020,"中 상하이서 ‘시진핑 퇴진, 공산당 퇴진’ 구호 나왔다",중국 베이징과 상하이 등 도시 곳곳에서 신종 코로나바이러스 감염증(코로나19) 봉쇄...,https://n.news.naver.com/mnews/article/020/000...
16,20221127,020,"“남욱 증언, 다른 공범과 일치해야 증거능력 인정 받을 것”",남욱 변호사가 21일 오전 서울 서초구 서울중앙지법에서 열린 대장동 개발 사업 로비...,https://n.news.naver.com/mnews/article/020/000...
17,20221127,020,28일 국토부-화물연대 첫 교섭…‘안전운임제 품목 확대’ 진통 예고,민주노총 공공운수노조 화물연대본부(화물연대)의 집단운송거부(총파업)가 나흘째 이어진...,https://n.news.naver.com/mnews/article/020/000...
18,20221127,020,비조합원 화물차에 쇠구슬 추정 물채 ‘쾅’…운전자 부상,ⓒ 뉴스1전국민주노동조합총연맹 화물연대본부의 총파업이 나흘째 이어지는 가운데 26일...,https://n.news.naver.com/mnews/article/020/000...
